In [1]:
%pip install -qU \
    replicate \
    langchain \
    sentence_transformers \
    pdf2image \
    pdfminer \
    pdfminer.six \
    unstructured


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ctransformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install faiss-cpu

In [4]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

In [7]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [5]:
!pip install --upgrade langchain
!pip install langchain_community

  Using cached langchain_core-0.2.0-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.2.0-py3-none-any.whl (307 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-anthropic 0.1.11 requires langchain-core<0.2.0,>=0.1.43, but you have langchain-core 0.2.0 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.0 which is incompatible.
langchain-experimental 0.0.58 requires langchain<0.2.0,>=0.1.17, but you have langchain 0.2.0 which is incompatible.
langchain-experimental 0.0.58 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.0 which is incompatible.
langchain-google-genai 0.0.6 requires langchain-core<0.2,>=0.1, but you have langchain-core 0.2.0 which is incompatible.
langchain-openai 0.0.6 requires langchain-core<0.2,>=0.1.16, but you have langchain-core 0.2.0 which is incompatible.
langchain-pinecone 0.1.0 requires langchain-core<0.2.0,>=0.1.40, but you have langchain-core 0

  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.0
    Uninstalling langchain-core-0.2.0:
      Successfully uninstalled langchain-core-0.2.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.0 requires langchain-core<0.3.0,>=0.2.0, but you have langchain-core 0.1.52 which is incompatible.
langchain-experimental 0.0.58 requires langchain<0.2.0,>=0.1.17, but you have langchain 0.2.0 which is incompatible.
langchain-text-splitters 0.2.0 requires langchain-core<0.3.0,>=0.2.0, but you have langchain-core 0.1.52 which is incompatible.


In [53]:
from langchain.llms import CTransformers

llama_model = CTransformers(
    model = "C:\\Users\\adity\\Downloads\\BioMistral-7B.Q4_K_M.gguf",
    model_type = "BioMistral",
    config = {'max_new_tokens':1000,
              'temperature':0.75,
              'context_length':2000}
)

In [ ]:
'''#load the external data source
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Medical_Gale_Encyclopedia.pdf")
documents = loader.load()
if len(documents) > 0:
  print("yes")
else:
  print("no")'''

no


In [27]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:\\Users\\adity\\Desktop\\Chat UI\\Data\\Medical_Gale.pdf")
documents = loader.load()

if len(documents) > 0:
    print("yes")
else:
    print("no")


yes


In [28]:
print(documents)

[Document(page_content='Description\nMany unexpected and fascinating events occur dur-\ning the development of a fetus inside the womb. Thestomach and intestines begin development outside thebaby’s abdomen and only later does the abdominal wallenclose them. Occasionally, either the umbilical openingis too large, or it develops improperly, allowing the bow-els or stomach to remain outside or squeeze through theabdominal wall.\nCauses and symptoms\nThere are many causes for birth defects that still\nremain unclear. Presently, the cause(s) of abdominal walldefects is unknown, and any symptoms the mother mayhave to indicate that the defects are present in the fetusare nondescript.\nDiagnosis\nAt birth, the problem is obvious, because the base of\nthe umbilical cord at the navel will bulge or, in worsecases, contain viscera (internal organs). Before birth, anultrasound examination may detect the problem. It isalways necessary in children with one birth defect to lookfor others, because birt

In [29]:
#Get text splits from document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [30]:
print("Document text has been split into", len(all_splits), "chunks.")

Document text has been split into 1049 chunks.


In [31]:
import torch

cuda_available = torch.cuda.is_available()
integrated_gpu_name = torch.cuda.get_device_name(0) if cuda_available else None

print("CUDA available:", cuda_available)
print("Integrated GPU name:", integrated_gpu_name)

CUDA available: False
Integrated GPU name: None


In [32]:
#Use the embedding model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2" # embedding model
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


In [33]:
# Use vector store to store embeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

try:
    if embeddings:
        # Assuming embeddings is a list of embeddings
        vectorstore = FAISS.from_documents(all_splits, embeddings)
    else:
        print("Embeddings list is empty. Please check your data.")
except TypeError:
    print("Please check your data.")


In [34]:
def md(t):
  display(Markdown(t))

In [54]:
#Question initialization
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []

In [ ]:
'''# Query 1
query = "what is frost bite ??"
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])'''

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Frostbite is a condition that occurs when the hands turn blue,  blue-red and then red before numbness, tingling, burning sensations occur. It results from poor blood flow to the extremities, which can be caused by cold temperatures or emotional stress. The disease often affects women and causes discomfort, with sweaty, cold, bluish hands and feet.

In [55]:
# Query 1
query = "How to cure stomach ache?"
result = chain.invoke({"question": query, "chat_history": chat_history})
md(result['answer'])

 There are several different kinds of drugs available for treatment of achalasia, which include pneumatic dilatation and laser myotomy. These procedures are usually combined with medical therapy using medications such as nitroglycerin, calcium channel blockers, and anticholinergic agents .

In [58]:
# Query 2 (1.1)
query = "how to stop nosebleeding"
res1 = chain.invoke({"question": query, "chat_history": chat_history})
md(res1['answer'])

 1. The simplest treatment for nosebleeds is to press firmly on the bleed-ing site with a cotton swab until the bleeding stops. If the bleeding continues, spray decongestant (e.g., Becher's) into both nostrils. This may stop the bleeding almost immed-iately. 2. Slight nosebleeds can usually be stopped by pinching the soft part of the septal wall below or between the nostrils. If the bleeding continues, spray decongestant (e.g., Becher's) into both nostrils; an ice bag is placed on the face, especially over the affected nostril.

In [60]:
# Query 2 (1.2)
query = "how to stop nosebleeding?"
res2 = chain.invoke({"question": query, "chat_history": chat_history})
md(res2['answer'])

 1) Pressure; 2) Cold water; 3) Elevation of head; 4) Direct pressure on nose; 5) Hot water; 6) Pineapple juice; 7) Cayenne pepper; 8) Vitamin K.

In [61]:
#General Query
query = input()
result2 = chain.invoke({"question": query, "chat_history": chat_history})
md(result2['answer'])

 The answer is type II Diabetes

In [62]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Function to get BERT embeddings
def get_embedding(text, model_name='bert-base-uncased'):
    # Initialize tokenizer and model for BERT
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Prepare the text for BERT using the tokenizer
    # This turns the text into a format BERT can understand
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Disable gradient calculation for performance
    with torch.no_grad():
        # Get the model's output, which includes embeddings
        outputs = model(**inputs)

    # The embeddings are averaged to get a single vector per input
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings.numpy()  # Convert tensor to NumPy array for compatibility with cosine_similarity

In [63]:
def compare_responses(response1, response2, model_name='bert-base-uncased'):
    # Convert responses to embeddings
    emb1 = get_embedding(response1, model_name)
    emb2 = get_embedding(response2, model_name)

    # Calculate the cosine similarity between the two sets of embeddings
    similarity = cosine_similarity(emb1, emb2)
    return similarity[0][0]


# Compare the two responses and print the similarity score
similarity_score = compare_responses(res1['answer'], res2['answer'])
print("Cosine similarity score:", similarity_score)

Cosine similarity score: 0.72517


In [64]:
!pip install gradio

In [45]:
import gradio as gr
from langchain.llms import CTransformers
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize document loader and load the PDFs from the specified directory
loader = PyPDFLoader('C:\\Users\\adity\\Desktop\\Chat UI\\Data\\Medical_Gale.pdf')
documents = loader.load()

# Process the loaded documents into manageable text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# Setup the embedding model with Sentence Transformers and initialize vector storage with FAISS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
vectorstore = FAISS.from_documents(all_splits, embeddings)

# Define the response function for the chatbot, which uses selected model and configuration
def chatbot_response(user_input, max_new_tokens, temperature, context_length):
    model = CTransformers(
        model="C:\\Users\\adity\\Downloads\\BioMistral-7B.Q4_K_M.gguf",
        model_type="mistral",
        config={
            'max_new_tokens': int(max_new_tokens),
            'temperature': float(temperature),
            'context_length': int(context_length)
        }
    )

    # Create a conversational retrieval chain with the selected model and query it
    chain = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever(), return_source_documents=True)
    chat_history = []
    result = chain({"question": user_input, "chat_history": chat_history})
    return result['answer']

# Setup the Gradio interface with dynamic inputs based on model type
interface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Textbox(lines=2, placeholder="Type your question here...", label="Your Question", interactive=True),
        gr.Slider(label="Max New Tokens", minimum=1, maximum=2000, value=1000, step=1, interactive=True),
        gr.Slider(label="Temperature", minimum=0.1, maximum=1.0, value=0.75, step=0.01, interactive=True),
        gr.Slider(label="Context Length", minimum=100, maximum=4000, value=2000, step=1, interactive=True)
    ],
    outputs=gr.Textbox(label="Chatbot Response", interactive=True),
    title="🩺 MEDICAL Chatbot",
    description="""
    <div style='text-align: center;'>
        <img src='https://img.lovepik.com/free-png/20220126/lovepik-medical-robot-png-image_401792159_wh1200.png' alt='Medical Bot' width='100'>
        <p>Type your question and get a response from the Medical Bot!</p>
    </div>
    """,
    theme="huggingface"
)

# Run the interface, making it available as a web application
interface.launch()


KeyboardInterrupt: 

In [ ]:
'''import gradio as gr
from langchain.llms import CTransformers
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os

# Define model identifiers with more descriptive names for easier selection in the UI
model_names = {
    "7B Model (LLaMa 2)": "llama-2-7b-chat",
    "Gemini Model": "gemini-model"
}

# Initialize document loader and load the PDFs from the specified directory
loader = PyPDFLoader('')
documents = loader.load()

# Process the loaded documents into manageable text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# Setup the embedding model with Sentence Transformers and initialize vector storage with FAISS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
vectorstore = FAISS.from_documents(all_splits, embeddings)

# Define the response function for the chatbot, which uses selected model and configuration
def chatbot_response(model_type, user_input, max_new_tokens, temperature, context_length):
    # Select the model based on user input and initialize with specified parameters
    if model_type == "7B Model (LLaMa 2)":
        model = CTransformers(
            model="C:\\Users\\adity\\Downloads\\llama-2-7b-chat.ggmlv3.q2_K.bin",
            model_type="llama",
            config={
                'max_new_tokens': int(max_new_tokens),
                'temperature': float(temperature),
                'context_length': int(context_length)
            }
        )
    elif model_type == "Gemini Model":
        # Example of initializing Gemini model with API key
        model = GeminiModel(api_key="", config={
            'max_new_tokens': int(max_new_tokens),
            'temperature': float(temperature),
            'context_length': int(context_length)
        })

    # Create a conversational retrieval chain with the selected model and query it
    chain = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever(), return_source_documents=True)
    chat_history = []
    result = chain({"question": user_input, "chat_history": chat_history})
    return result['answer']

# Setup the Gradio interface with dynamic inputs based on model type
interface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Radio(list(model_names.keys()), label="Model Type"),
        gr.Textbox(lines=2, placeholder="Type your question here..."),
        gr.Number(label="Max New Tokens", value=1000, step=1),
        gr.Number(label="Temperature", value=0.75, step=0.01),
        gr.Number(label="Context Length", value=2000, step=1)
    ],
    outputs="text",
    title="MEDICAL Chatbot",
    description="Select the model type and adjust the parameters to see how they affect the model's responses."
)

# Run the interface, making it available as a web application
interface.launch()'''
